Para el programa se necesitará importar las siguientes librerías que se instalan de la siguiente manera:

BeautifulSoup: para recorrer un html

pip install BeautifulSoup4

selenium: Para ingresar a sitios webs desarrollados con scripts

pip install selenium

lxml (parser): Para poder acceder al html del sitio web.

pip install lxml

In [4]:
from bs4 import BeautifulSoup 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import re
from time import gmtime, strftime
import sqlite3


Para que funcione el programa hay que descargar el driver de edge, que se encuentra en https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/

Hay que descargar la versión que se encuentra en tu windows.

Recomiendo copiar el driver en una carpeta llamada "drivers" en la carpeta de tu programa.
driver = webdriver.Edge("localización del driver en tu sistema",options=option)

In [44]:
def get_twitter_trends_to_dict(driver):

    content=driver.page_source.encode('utf-8').strip()

    ## se utiliza BeautifulSoup para tomar el html
    soup = BeautifulSoup(content,"lxml")

    ## Se buscan todas las eticatas span con el nombre de clase donde está la información que necesitamos
    tops= soup.find_all("div", class_ = "css-1dbjc4n r-1d09ksm r-18u37iz r-1wbh5a2")
    tittles=soup.find_all("div", class_="css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-b88u0q r-rjixqe r-1bymd8e r-bcqeeo r-qvutc0") 
    twits=soup.find_all("div", class_="css-901oao r-14j79pv r-37j5jr r-n6v787 r-16dba41 r-1cwl3u0 r-14gqq1x r-bcqeeo r-qvutc0")

    trending_toppic_dict=[]

    #se divide la lista en cada una de las etiquetas, tomando unicamente el texto de la misma.
    for i in range(len(twits)):
        trend={}

        trend["top"]= int(re.findall('[0-9]+',tops[i].text)[0])

        trend["tittle"]=tittles[i].text

        if re.search("[0-9]",twits[i].text):
            if re.search("K",twits[i].text):
                try:
                    trend["twits"]= int(re.findall('[0-9]+',twits[i].text)[0]+re.findall('[0-9]+',twits[i].text)[1])*1000
                except:
                    trend["twits"]= int(re.findall('[0-9]+',twits[i].text)[0])*1000

            else:
                try:
                    trend["twits"]= int(re.findall('[0-9]+',twits[i].text)[0]+re.findall('[0-9]+',twits[i].text)[1])
                except:
                    trend["twits"]= int(re.findall('[0-9]+',twits[i].text)[0])
        
        trend["date"]= strftime("%Y-%m-%d", gmtime())
        trend["time"]=  strftime("%H:%M:%S", gmtime())
        
        trending_toppic_dict.append(trend)
           

    return trending_toppic_dict


def trending_toppic ():
     ## optiones para no abrir la ventana al ejecutar el programa
    option = webdriver.EdgeOptions()
    option.add_argument('--headless')

    ##ejecusión del driver, si se quita el parámetro "options" se abrirá la ventana del browser
    driver = webdriver.Edge("./drivers\msedgedriver.exe", options=option)

    ##Se toma el url de la página a scriptar y se determina el código de caracteres, para el español sería utf-8
    url = driver.get('https://twitter.com/explore/tabs/trending')

    sleep(5)

    trending_toppic = get_twitter_trends_to_dict(driver)

    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

        trending_toppic_2=get_twitter_trends_to_dict(driver)
    
    
    for i in range(len(trending_toppic)):
        j=trending_toppic_2[i]["top"]
        
        if j-i-1>= len(trending_toppic):
            break
        if trending_toppic[j-i-1]["top"]==trending_toppic_2[0]["top"]:
            del trending_toppic_2[0]

    trending_toppic+=trending_toppic_2
    
    return trending_toppic
    
trending_toppic ()

# Alumno Miguel Rodriguez Saquilan 13/11/2022 "Base de datos"


class tt_twitter:

    def __init__(self):
        self.con = sqlite3.connect("database.db")
        self.cur = self.con.cursor()
        self.create_table()

    def create_table(self):
        #self.cur.execute("""DROP TABLE tt_twitter""")
        self.cur.execute("""CREATE TABLE IF NOT EXISTS tt_twitter (
        top INTEGER,
        tittles TEXT,
        twitts INTEGER,
        date TEXT,
        time TEXT
        )""")

    def insert(self,item):
        self.cur.execute("""INSERT OR IGNORE INTO tt_twitter VALUES(?,?,?,?,?)""",item)
        self.con.commit()

    def read(self):
        self.cur.execute("""SELECT * FROM tt_twitter""")
        rows = self.cur.fetchall()
        return rows



C:\Users\nacho\AppData\Local\Temp\ipykernel_4596\2381802557.py:51: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge("./drivers\msedgedriver.exe", options=option)


[{'top': 1,
  'tittle': '#BuenMiercoles',
  'twits': 127000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 2,
  'tittle': 'Corte',
  'twits': 366000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 3,
  'tittle': '#GH2002',
  'twits': 56000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 4,
  'tittle': 'Sadio Mané',
  'twits': 343000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 5,
  'tittle': 'Consejo de la Magistratura',
  'twits': 273000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 6,
  'tittle': 'Lo Celso',
  'twits': 6072,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 7,
  'tittle': '#GranHernano2022',
  'twits': 725000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 8,
  'tittle': 'Tora',
  'twits': 236000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 9,
  'tittle': 'Luis Juez',
  'twits': 235000,
  'date': '2022-11-09',
  'time': '13:05:27'},
 {'top': 10,
  'tittle': 'Qatar',
  'twits': 17000,
  'dat